In [1]:
import ll

ll.pretty()

TensorBoard/TensorBoardX not found. Disabling TensorBoardLogger. Please install TensorBoard with `pip install tensorboard` or TensorBoardX with `pip install tensorboardx` to enable TensorBoard logging.


In [2]:
from pathlib import Path

base_path = Path("/mnt/datasets/mptrj-mace/mptrj-gga-ggapu/")
base_path

Path('/mnt/datasets/mptrj-mace/mptrj-gga-ggapu')

In [3]:
extxyz_files = list(base_path.glob("**/*.extxyz"))
print(len(extxyz_files))

145923


In [4]:
import ase.io

f = extxyz_files[51]
atoms_list = ase.io.read(f, index=":")
print(type(atoms_list), len(atoms_list))

<class 'list'> 7


In [5]:
import rich


rich.print(atoms_list[0].todict())
rich.print(atoms_list[5].todict())

{
    'numbers': array[20] i64 x∈[6, 83] μ=22.600 σ=30.210,
    'positions': array[20, 3] n=60 x∈[0.100, 7.276] μ=3.242 σ=1.983,
    'forces': array[20, 3] n=60 x∈[-0.161, 0.161] μ=5.782e-19 σ=0.059,
    'magmoms': array[20] all_zeros,
    'cell': array[3, 3] n=9 x∈[-0.750, 8.602] μ=2.162 σ=3.470 [[6.960, -0.033, -0.499], [-0.032, 5.214, -0.004], 
[-0.750, -0.002, 8.602]],
    'pbc': array[3] bool x∈[True, True] μ=1.000 σ=0. [True, True, True],
    'info': {
        'energy': -135.80326564,
        'stress': array[3, 3] n=9 x∈[-0.005, 0.002] μ=-0.001 σ=0.002 [[-0.003, 0.000, 0.002], [0.000, -0.005, 
0.000], [0.002, 0.000, -0.003]],
        'bandgap': 1.5052000000000003,
        'e_per_atom_relaxed': -7.2055584999999995,
        'mp_id': 'mp-1178485',
        'energy_per_atom': -7.202363282,
        'ef_per_atom_relaxed': -7.2055584999999995,
        'corrected_total_energy': -144.04726564,
        'ef_per_atom': -1.61095276075,
        'task_id': 'mp-1178485',
        'calc_id': 0,
        'ionic_step': 4
    }
}

{
    'numbers': array[20] i64 x∈[6, 83] μ=22.600 σ=30.210,
    'positions': array[20, 3] n=60 x∈[0.106, 7.303] μ=3.261 σ=1.987,
    'forces': array[20, 3] n=60 x∈[-0.779, 0.779] μ=-4.626e-18 σ=0.298,
    'cell': array[3, 3] n=9 x∈[-0.726, 8.623] μ=2.174 σ=3.474 [[6.942, -0.028, -0.483], [-0.028, 5.278, -0.006], 
[-0.726, -0.007, 8.623]],
    'pbc': array[3] bool x∈[True, True] μ=1.000 σ=0. [True, True, True],
    'info': {
        'energy': -135.72813107,
        'stress': array[3, 3] n=9 x∈[-0.005, 0.025] μ=0.004 σ=0.010 [[4.956e-05, 0.000, -0.005], [0.000, 0.025, 
-0.000], [-0.005, -0.000, 0.017]],
        'e_per_atom_relaxed': -7.2055584999999995,
        'mp_id': 'mp-1178485',
        'energy_per_atom': -7.1986065534999994,
        'ef_per_atom_relaxed': -7.2055584999999995,
        'corrected_total_energy': -143.97213107,
        'ef_per_atom': -1.6071960322499992,
        'task_id': 'mp-1178485',
        'calc_id': 1,
        'ionic_step': 0
    }
}

In [7]:
from collections.abc import Sequence
import datasets
from einops import rearrange


def atoms_to_dict(atoms):
    d = atoms.todict()
    info = d.pop("info")
    return {**d, **info}


def generator(file_path: Path):
    atoms_list = ase.io.read(file_path, index=":")
    assert isinstance(
        atoms_list, Sequence
    ), f"atoms_list is not a sequence: {type(atoms_list)}"

    for i, atoms in enumerate(atoms_list):
        d = atoms_to_dict(atoms)
        d["filename"] = str(file_path.stem)
        d["extxyz_id"] = i

        # Pop magmom as it is not present in all files
        d.pop("magmoms", None)
        d.pop("bandgap", None)

        assert d["forces"].ndim == 2
        assert d["positions"].ndim == 2
        assert d["cell"].ndim == 2
        assert d["stress"].ndim == 2

        d["num_atoms"] = d["numbers"].shape[0]

        yield d


def generator_all(shards: list[Path]):
    for file_path in shards:
        yield from generator(file_path)


dataset = datasets.Dataset.from_generator(
    generator_all,
    gen_kwargs={"shards": extxyz_files},
    num_proc=32,
    features=datasets.Features(
        {
            # "numbers": datasets.Array2D(shape=(None, 1), dtype="int64"),
            "numbers": datasets.Sequence(datasets.Value("int64")),
            # "positions": datasets.Array2D(shape=(None, 3), dtype="float32"),
            "positions": datasets.Sequence(
                datasets.Sequence(datasets.Value("float32"))
            ),
            # "forces": datasets.Array2D(shape=(None, 3), dtype="float32"),
            "forces": datasets.Sequence(datasets.Sequence(datasets.Value("float32"))),
            "cell": datasets.Array2D(shape=(3, 3), dtype="float32"),
            "pbc": datasets.Sequence(datasets.Value("bool")),
            "energy": datasets.Value("float32"),
            "stress": datasets.Array2D(shape=(3, 3), dtype="float32"),
            "e_per_atom_relaxed": datasets.Value("float32"),
            "mp_id": datasets.Value("string"),
            "energy_per_atom": datasets.Value("float32"),
            "ef_per_atom_relaxed": datasets.Value("float32"),
            "corrected_total_energy": datasets.Value("float32"),
            "ef_per_atom": datasets.Value("float32"),
            "task_id": datasets.Value("string"),
            "calc_id": datasets.Value("int64"),
            "ionic_step": datasets.Value("int64"),
            "filename": datasets.Value("string"),
            "extxyz_id": datasets.Value("int64"),
            "num_atoms": datasets.Value("int64"),
        }
    ),
)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/32 [00:00<?, ?it/s]

Dataset({
    features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 'ionic_step', 'filename', 'extxyz_id', 'num_atoms'],
    num_rows: 1580395
})

In [18]:
d_test = dataset.train_test_split(test_size=0.0065)
train, test = d_test["train"], d_test["test"]

d_val = train.train_test_split(test_size=0.0065)
train, val = d_val["train"], d_val["test"]

rich.print({"train": train, "val": val, "test": test})

{
    'train': Dataset({
    features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 
'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 
'ionic_step', 'filename', 'extxyz_id', 'num_atoms'],
    num_rows: 1559916
}),
    'val': Dataset({
    features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 
'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 
'ionic_step', 'filename', 'extxyz_id', 'num_atoms'],
    num_rows: 10206
}),
    'test': Dataset({
    features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 
'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 
'ionic_step', 'filename', 'extxyz_id', 'num_atoms'],
    num_rows: 10273
})
}

In [20]:
ddict = datasets.DatasetDict({"train": train, "val": val, "test": test})
rich.print(ddict)

ddict.push_to_hub("nimashoghi/mptrj", private=True)

DatasetDict({
    train: Dataset({
        features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 
'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 
'ionic_step', 'filename', 'extxyz_id', 'num_atoms'],
        num_rows: 1559916
    })
    val: Dataset({
        features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 
'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 
'ionic_step', 'filename', 'extxyz_id', 'num_atoms'],
        num_rows: 10206
    })
    test: Dataset({
        features: ['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 
'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 
'ionic_step', 'filename', 'extxyz_id', 'num_atoms'],
        num_rows: 10273
    })
})

Uploading the dataset shards:   0%|          | 0/5 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/nimashoghi/mptrj/commit/e75795f0c9f0c0ed44bcf8012689f5fc180c19d4', commit_message='Upload dataset', commit_description='', oid='e75795f0c9f0c0ed44bcf8012689f5fc180c19d4', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
# Split dataset into train, validation and test
train, test = dataset.train_test_split(test_size=0.1)
train, val = train.train_test_split(test_size=0.1)

rich.print({"train": train, "val": val, "test": test})

AttributeError: 'str' object has no attribute 'train_test_split'

In [ ]:
dt = dataset.with_format("torch")
rich.print(dt[0])

rich.print(dt.features)

In [ ]:
dataset.push_to_hub("nimashoghi/mptrj", private=True)